<a href="https://colab.research.google.com/github/HarshaVardhanLanka/OpenDeepResearchAgent/blob/main/Day_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**In LangGraph, the Searcher is a standalone module. If the Searcher fails, the Planner and Writer don't care—they just wait for the state to update.**

**Phase 1: The Setup**

We need to install langgraph and the official LangChain integration for Gemini (langchain-google-genai)

In [ ]:
!pip install -q -U langgraph langchain langchain-google-genai tavily-python pypdf

**Phase 2: Config & State Definition**

Here we define the "Shared Memory" (State) that will be passed between agents.

In [ ]:
import os
from google.colab import userdata
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

# LangChain Imports
from langchain_google_genai import ChatGoogleGenerativeAI
from tavily import TavilyClient

# 1. Setup API Keys
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY') # Note: Ensure this matches your Secret name
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
except:
    print("⚠️ Error: Please set GOOGLE_API_KEY and TAVILY in Colab Secrets.")

# 2. Initialize the LLM
# We use 'gemini-1.5-flash' as it is fast and efficient for agent workflows
llm = ChatGoogleGenerativeAI(model="gemini-pro-latest", temperature=0)

# 3. Initialize Tavily
tavily = TavilyClient(api_key=TAVILY_API_KEY)

print("Setup complete.")

Setup complete.


In [ ]:
# 4. DEFINE THE STATE (The "Clipboard")
# This is the data structure passed between agents
class AgentState(TypedDict):
    topic: str                # The User's input (or PDF content)
    research_plan: List[str]  # Output from Planner
    search_results: str       # Output from Searcher
    final_report: str         # Output from Writer

# **Memory System**

In [ ]:
import os
import json
import datetime

# --- MEMORY SYSTEM ---
class HistoryManager:
    def __init__(self):
        self.history_file = "agent_history.json"
        self.history = self.load_history()

    def load_history(self):
        """Loads history from a JSON file if it exists."""
        if os.path.exists(self.history_file):
            with open(self.history_file, 'r') as f:
                return json.load(f)
        return []

    def save_entry(self, input_text, mode, final_report):
        """Saves a new research session to memory."""
        entry = {
            "id": len(self.history) + 1,
            "timestamp": str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")),
            "mode": mode,
            "input": input_text[:50] + "..." if len(input_text) > 50 else input_text, # Preview only
            "full_input": input_text,
            "report": final_report
        }
        self.history.append(entry)
        with open(self.history_file, 'w') as f:
            json.dump(self.history, f)
        print("✅ Research saved to History.")

    def show_history(self):
        """Displays list of past inputs."""
        if not self.history:
            print("\n📭 History is empty.")
            return None

        print("\n📚 RESEARCH HISTORY:")
        print(f"{'ID':<4} | {'Time':<18} | {'Mode':<8} | {'Topic/Input'}")
        print("-" * 60)
        for item in self.history:
            print(f"{item['id']:<4} | {item['timestamp']:<18} | {item['mode']:<8} | {item['input']}")
        return self.history
# Initialize Memory
memory = HistoryManager()

**Phase 3: The Agents (Nodes)**

We will define your three agents as specific functions. Each function receives the state, does work, and returns an update to the state.

**1. The Planner Agent**

Breaks the topic down.

In [ ]:
def planner_node(state: AgentState):
    print(f"🧠 [Planner]: Analying request...")
    topic = state['topic']

    # We ask the LLM to return a clean list
    prompt = f"""
    You are a Research Planner.
    Topic/Context: {topic}

    Task: Generate 3 distinct, specific search queries to gather comprehensive information.
    Constraint: Return ONLY the 3 queries separated by newlines. Do not number them.
    """

    response = llm.invoke(prompt)

    # Process output into a list
    queries = [q.strip() for q in response.content.split('\n') if q.strip()]
    print(f"   -> Generated Plan: {queries[:3]}")

    return {"research_plan": queries[:3]}

**2. The Searcher Agent**

Executes the plan.

In [ ]:
def searcher_node(state: AgentState):
    print(f"🔎 [Searcher]: Executing research plan...")
    queries = state['research_plan']
    results = []

    for q in queries:
        print(f"   - Searching: {q}")
        try:
            # search_depth="advanced" gives better quality for reports
            response = tavily.search(query=q, max_results=1, search_depth="basic")
            if response['results']:
                content = response['results'][0]['content']
                results.append(f"Source: {q}\nContent: {content}")
            else:
                results.append(f"Source: {q}\nContent: No data found.")
        except Exception as e:
            print(f"Error searching {q}: {e}")

    # Combine all results into one string
    combined_content = "\n\n".join(results)
    return {"search_results": combined_content}

**3. The Writer Agent**

Synthesizes the report

In [ ]:
def writer_node(state: AgentState):
    print(f"✍️ [Writer]: Synthesizing final report...")

    topic = state['topic']
    data = state['search_results']

    prompt = f"""
    You are a Senior Technical Writer.

    Original Topic/Context: {topic[:500]}... (truncated for brevity)

    Verified Research Data:
    {data}

    Task: Write a professional, structured summary.
    1. Introduction
    2. Key Findings (incorporate the research data)
    3. Conclusion

    Format: Markdown.
    """

    response = llm.invoke(prompt)
    return {"final_report": response.content}

**Phase 4: The Execution Pipeline (Graph)**

This is where we wire the agents together using LangGraph.

In [ ]:
# 1. Initialize Graph
workflow = StateGraph(AgentState)

# 2. Add Nodes
workflow.add_node("planner", planner_node)
workflow.add_node("searcher", searcher_node)
workflow.add_node("writer", writer_node)

# 3. Define Edges (The Flow)
workflow.set_entry_point("planner")       # User Input -> Planner
workflow.add_edge("planner", "searcher")  # Planner -> Searcher
workflow.add_edge("searcher", "writer")   # Searcher -> Writer
workflow.add_edge("writer", END)          # Writer -> Finish

# 4. Compile
app = workflow.compile()
print("✅ Multi-Agent System Compiled successfully.")

✅ Multi-Agent System Compiled successfully.


**Phase 5: Main Execution Loop (Integration)**

We will keep your existing PDF and History logic, but now we feed the data into the Graph (app.invoke) instead of calling functions manually.

In [ ]:
import io
import pypdf
from google.colab import files
from IPython.display import display, Markdown

# --- HELPER: PDF EXTRACTOR ---
def extract_pdf_text(uploaded_file):
    pdf_reader = pypdf.PdfReader(io.BytesIO(uploaded_file))
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text() + "\n"
    return text

# --- MAIN APP ---
def main():
    print("🤖 LANGGRAPH RESEARCH AGENT INITIALIZED")

    while True:
        print("\n" + "="*15 + " STRICT HYBRID AGENT " + "="*15)
        print("1. Research a Topic (Text 💬)")
        print("2. Research a Paper (Upload PDF 📖)")
        print("3. View History 📜")
        print("4. Exit 📤")

        choice = input("Select Option: ")

        if choice == '4':
            print("Goodbye.👋")
            break

        # --- OPTION 3: HISTORY ---
        if choice == '3':
            history_data = memory.show_history()
            if history_data:
                sub_choice = input("\nEnter ID to view full report (or Press Enter to go back): ")
                if sub_choice.isdigit():
                    idx = int(sub_choice) - 1
                    if 0 <= idx < len(history_data):
                        entry = history_data[idx]
                        print(f"\n🔁 RECALLING REPORT FOR: {entry['input']}")
                        print("-" * 50)
                        display(Markdown(entry['report']))
                        input("\nPress Enter to continue...")
            continue # Skip the rest and go back to menu

        # SETUP INPUT
        mode = ""
        input_data = ""
        label = "" # Used for history title

        # Prepare Input
        user_input = ""

        if choice == '1':
            user_input = input("Enter Topic: ")
            mode = "Text"
            label = user_input
        elif choice == '2':
            print("Upload PDF:")
            uploaded = files.upload()
            if uploaded:
                fn = next(iter(uploaded))
                raw_text = extract_pdf_text(uploaded[fn])
                # We wrap the PDF text in a prompt so the Planner knows what to do
                user_input = f"Analyze and validate this paper content: {raw_text[:10000]}"
                mode = "PDF"
                label = fn
            else:
                print("No file.")
                continue
        else:
            print("Invalid option. Please try again.")
            continue

        if user_input:
            print("\n🚀 STARTING AGENT PIPELINE...\n")

            # --- THE MAGIC HAPPENS HERE ---
            # We invoke the LangGraph application
            final_state = app.invoke({"topic": user_input})

            print("\n" + "="*50)
            print("📊 FINAL REPORT")
            print("="*50 + "\n")
            display(Markdown(final_state['final_report']))
            memory.save_entry(label, mode, final_state['final_report'])

if __name__ == "__main__":
    main()

🤖 LANGGRAPH RESEARCH AGENT INITIALIZED

=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 3

📚 RESEARCH HISTORY:
ID   | Time               | Mode     | Topic/Input
------------------------------------------------------------
1    | 2025-11-28 13:48   | PDF      | 2511.17546v1 (1).pdf
2    | 2025-11-28 13:51   | Text     | ocular toxoplasmosis

Enter ID to view full report (or Press Enter to go back): 1

🔁 RECALLING REPORT FOR: 2511.17546v1 (1).pdf
--------------------------------------------------


Of course. Here is a professional, structured summary based on the provided information.

***

## Technical Summary: Analysis of "Reference Quadrupole Moments of Transition Elements from Lamb Shifts in Muonic Atoms"

### 1. Introduction

The paper by S. Rathi et al. presents a novel method for determining reference values for the nuclear electric quadrupole moments (Q) of transition elements. The methodology is centered on the high-precision measurement of the Lamb shift in muonic atoms. A muonic atom, where an electron is replaced by a much heavier muon, exhibits significantly enhanced sensitivity to nuclear structure effects due to the muon's proximity to the nucleus. By precisely measuring the energy of X-rays emitted during muonic transitions, the researchers can isolate the Lamb shift, which contains contributions from nuclear properties, including the quadrupole moment. This work aims to establish a new set of highly accurate, reference-grade Q values that can be used to benchmark and validate nuclear theories.

### 2. Key Findings

The analysis and validation of the paper's content, supported by related research data, highlight several key findings:

*   **Advanced Experimental Feasibility:** The precision required to measure subtle energy shifts in muonic X-rays is a significant experimental challenge. The success of this method is underpinned by the availability of state-of-the-art instrumentation. As verified by related research, the development of **high-resolution cryogenic microcalorimeters** provides the necessary energy-dispersive spectroscopy (EDS) performance in the relevant 10-40 keV range, making the high-fidelity measurements proposed by Rathi et al. experimentally viable.

*   **Rigorous Treatment of Theoretical Uncertainties:** Extracting the quadrupole moment from the Lamb shift is not a direct measurement; it requires a sophisticated theoretical framework that accounts for multiple contributing effects. The paper's methodology must address known **theoretical uncertainties, particularly the nuclear polarization effect**, which arises from the nucleus being deformed by the muon's strong electric field. This correction is critical for achieving the high accuracy needed for a "reference" value.

*   **Establishment of a New Benchmark:** The work provides an independent and robust method for determining Q values, which is crucial for cross-validation within the field. The results are directly comparable to those from other established techniques for **determining reference electric quadrupole moments in open d-shell transition elements (Z=23-30)**. By providing a new dataset derived from a fundamentally different physical interaction (muon-nucleus vs. electron-nucleus), this research offers a critical benchmark to resolve discrepancies and improve the accuracy of nuclear models.

### 3. Conclusion

The paper by Rathi et al. represents a significant contribution to nuclear structure physics. By leveraging the unique properties of muonic atoms, the authors have developed a powerful technique for determining reference nuclear quadrupole moments. The validity of their approach is supported by parallel advancements in both experimental technology (high-resolution detectors) and theoretical modeling (corrections for nuclear polarization). The resulting Q values for transition elements serve as a crucial independent reference, enhancing the scientific community's ability to test and refine theoretical models of the atomic nucleus. This work exemplifies the synergy between precision measurement, advanced instrumentation, and rigorous theoretical analysis.


Press Enter to continue...

=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 3

📚 RESEARCH HISTORY:
ID   | Time               | Mode     | Topic/Input
------------------------------------------------------------
1    | 2025-11-28 13:48   | PDF      | 2511.17546v1 (1).pdf
2    | 2025-11-28 13:51   | Text     | ocular toxoplasmosis

Enter ID to view full report (or Press Enter to go back): 2

🔁 RECALLING REPORT FOR: ocular toxoplasmosis
--------------------------------------------------


Of course. Here is a professional summary based on the provided research data, structured as requested.

---

### **Subject: Summary on Ocular Toxoplasmosis (OT) Pathophysiology and Risk Factors**

### 1. Introduction

Ocular toxoplasmosis (OT) is a significant cause of infectious posterior uveitis, resulting from an inflammatory response to the parasite *Toxoplasma gondii*. The condition classically manifests as a focal, full-thickness necrotizing retinochoroiditis. Understanding the clinical characteristics and risk factors associated with recurrence and visual impairment is critical for patient prognosis and management. This summary outlines key findings from recent research regarding the disease's pathology and the primary predictors of adverse outcomes.

### 2. Key Findings

Analysis of recent data highlights specific factors that significantly increase the risk of disease recurrence and permanent vision loss.

#### **Pathophysiological Characteristics**
*   **Inflammatory Mechanism:** OT is characterized by a potent inflammatory response within the eye, leading to necrotizing retinochoroiditis. This process involves the death of retinal tissue, often occurring adjacent to a pre-existing chorioretinal scar from a previous infection.

#### **Risk Factors for Recurrence**
A 2021 meta-analysis identified two primary factors associated with a higher likelihood of disease recurrence:
*   **Patient Age:** Elderly patients (≥ 65 years) have a statistically significant, albeit slightly increased, risk of recurrence (OR: 1.02; *P* = 0.0493).
*   **Initial Presentation:** Patients presenting with active disease are nearly five times more likely to experience a recurrence compared to those without (OR: 4.74; *P* = 0.0011).

#### **Predictors of Blindness and Severe Visual Impairment**
The risk of blindness is strongly correlated with specific characteristics of the lesion and clinical presentation.

*   **Factors Related to Blindness During Active OT Presentation:**
    *   **Macular Involvement:** Lesions localized to the macula dramatically increase the risk of blindness (OR: 9.95; *P* = 0.0019).
    *   **Presence of Complications:** The development of any complication is the strongest predictor of blindness during an active episode (OR: 10.26; *P* < .0001).
    *   **Atypical Presentation:** Cases of atypical OT are associated with a significantly higher risk of blindness (OR: 4.99; *P* = 0.033).

*   **Factors Related to Blindness from Inactive OT Lesions:**
    *   **Macular Involvement:** The location of a lesion in the macula remains a critical risk factor even after the disease becomes inactive (OR: 9.95; *P* < .0001).
    *   **Lesion Size:** Inactive lesions larger than one-disc diameter (>1DD) are strongly associated with subsequent blindness (OR: 6.30; *P* = 0.0013).

### 3. Conclusion

Ocular toxoplasmosis is a destructive inflammatory condition where prognosis is heavily influenced by identifiable risk factors. The location and size of the retinochoroidal lesion, particularly macular involvement, are paramount predictors of severe visual impairment in both active and inactive disease states. Furthermore, the presence of active disease at presentation and the development of complications are critical indicators for the risk of recurrence and blindness, respectively. These findings underscore the importance of early diagnosis and careful monitoring to mitigate the risk of irreversible vision loss.


Press Enter to continue...

=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 3

📚 RESEARCH HISTORY:
ID   | Time               | Mode     | Topic/Input
------------------------------------------------------------
1    | 2025-11-28 13:48   | PDF      | 2511.17546v1 (1).pdf
2    | 2025-11-28 13:51   | Text     | ocular toxoplasmosis

Enter ID to view full report (or Press Enter to go back): 1

🔁 RECALLING REPORT FOR: 2511.17546v1 (1).pdf
--------------------------------------------------


Of course. Here is a professional, structured summary based on the provided information.

***

## Technical Summary: Analysis of "Reference Quadrupole Moments of Transition Elements from Lamb Shifts in Muonic Atoms"

### 1. Introduction

The paper by S. Rathi et al. presents a novel method for determining reference values for the nuclear electric quadrupole moments (Q) of transition elements. The methodology is centered on the high-precision measurement of the Lamb shift in muonic atoms. A muonic atom, where an electron is replaced by a much heavier muon, exhibits significantly enhanced sensitivity to nuclear structure effects due to the muon's proximity to the nucleus. By precisely measuring the energy of X-rays emitted during muonic transitions, the researchers can isolate the Lamb shift, which contains contributions from nuclear properties, including the quadrupole moment. This work aims to establish a new set of highly accurate, reference-grade Q values that can be used to benchmark and validate nuclear theories.

### 2. Key Findings

The analysis and validation of the paper's content, supported by related research data, highlight several key findings:

*   **Advanced Experimental Feasibility:** The precision required to measure subtle energy shifts in muonic X-rays is a significant experimental challenge. The success of this method is underpinned by the availability of state-of-the-art instrumentation. As verified by related research, the development of **high-resolution cryogenic microcalorimeters** provides the necessary energy-dispersive spectroscopy (EDS) performance in the relevant 10-40 keV range, making the high-fidelity measurements proposed by Rathi et al. experimentally viable.

*   **Rigorous Treatment of Theoretical Uncertainties:** Extracting the quadrupole moment from the Lamb shift is not a direct measurement; it requires a sophisticated theoretical framework that accounts for multiple contributing effects. The paper's methodology must address known **theoretical uncertainties, particularly the nuclear polarization effect**, which arises from the nucleus being deformed by the muon's strong electric field. This correction is critical for achieving the high accuracy needed for a "reference" value.

*   **Establishment of a New Benchmark:** The work provides an independent and robust method for determining Q values, which is crucial for cross-validation within the field. The results are directly comparable to those from other established techniques for **determining reference electric quadrupole moments in open d-shell transition elements (Z=23-30)**. By providing a new dataset derived from a fundamentally different physical interaction (muon-nucleus vs. electron-nucleus), this research offers a critical benchmark to resolve discrepancies and improve the accuracy of nuclear models.

### 3. Conclusion

The paper by Rathi et al. represents a significant contribution to nuclear structure physics. By leveraging the unique properties of muonic atoms, the authors have developed a powerful technique for determining reference nuclear quadrupole moments. The validity of their approach is supported by parallel advancements in both experimental technology (high-resolution detectors) and theoretical modeling (corrections for nuclear polarization). The resulting Q values for transition elements serve as a crucial independent reference, enhancing the scientific community's ability to test and refine theoretical models of the atomic nucleus. This work exemplifies the synergy between precision measurement, advanced instrumentation, and rigorous theoretical analysis.


Press Enter to continue...

=============== STRICT HYBRID AGENT ===============
1. Research a Topic (Text 💬)
2. Research a Paper (Upload PDF 📖)
3. View History 📜
4. Exit 📤
Select Option: 4
Goodbye.👋
